In [ ]:
# optional: make cells display wider
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# if running this notebook in the cloud, first clone and install the yuca repo

#! git clone https://github.com/rivesunder/yuca yuca
#%cd yuca
#! pip install -e .

# in colab, you'll need to restart the runtime after installing yuca

In [ ]:
import yuca
from yuca.multiverse import CA
from yuca.code import CODE
from yuca.patterns import get_orbium
from yuca.utils import seed_all

import torch
import numpy as np

import skimage
import skimage.io as sio
import skimage.transform

from copy import deepcopy

import matplotlib

import matplotlib.pyplot as plt
my_cmap = plt.get_cmap("magma")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['animation.embed_limit'] = 1024

import IPython

import yuca.analyze

from yuca.zoo.librarian import Librarian


def plot_grids(slow_grid, grid, fast_grid, my_cmap=plt.get_cmap("magma"), titles=["Step size demo"], vmin=0.0, vmax=1):

    global subplot_0
    global subplot_1
    global subplot_2

    if titles == None:
        titles = ["Title!"]

    fig, ax = plt.subplots(1,3, figsize=(4.5,1.5), facecolor="white")

    # TODO invert cmap
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0 = ax[0].imshow(slow_grid_display, interpolation="nearest")
    subplot_1 = ax[1].imshow(grid_display, interpolation="nearest")
    subplot_2 = ax[2].imshow(fast_grid_display, interpolation="nearest")
    
    fig.suptitle(titles[0], fontsize=8)
    for hh in range(3):
        ax[hh].set_yticklabels('')
        ax[hh].set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global subplot_1    
    global subplot_2
    global slow_grid
    global grid
    global fast_grid
    global ax
    global speedup_factor
    
    dt = 1.0 *ca.dt
    
    for ii in range(speedup_factor):
        grid = ca(grid)

        ca.dt = dt / 3.
        slow_grid = ca(slow_grid)
        ca.dt = min([dt * 3, 1.0])
        fast_grid = ca(fast_grid)
        ca.dt = dt
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0.set_array(slow_grid_display)
    subplot_1.set_array(grid_display)
    subplot_2.set_array(fast_grid_display)
    
    ax[0].set_title(f"Step size: {dt/3:.4f}", fontsize=7)
    ax[1].set_title(f"Step size: {dt:.4f}", fontsize=7)
    ax[2].set_title(f"Step size: {dt*3:.4f}", fontsize=7)
    
    plt.tight_layout()
    
def update_fig_precision(ii):

    global subplot_0    
    global subplot_1    
    global subplot_2
    global slow_grid
    global grid
    global fast_grid
    global ax
    global speedup_factor
    global scale_factor    
    
    dt = 1.0 *ca.dt
    
    default_ca_config = ca.make_config()
    default_radius = default_ca_config["neighborhood_kernel_config"]["radius"]
    
    big_ca_config = deepcopy(default_ca_config)
    small_ca_config = deepcopy(default_ca_config)
    
    big_ca = CA()
    big_ca.load_config(big_ca_config)
    big_ca.no_grad()
    small_ca = CA()
    small_ca.load_config(small_ca_config)
    small_ca.no_grad()
    
    small_ca.dt = dt
    big_ca.dt = dt
    
    big_ca.to(torch.float64) 
    #small_ca.to(torch.float16)
        
    for jj in range(speedup_factor):
        grid = ca(grid)
        slow_grid = big_ca(slow_grid)
        fast_grid = small_ca(fast_grid)
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0.set_array(slow_grid_display)
    subplot_1.set_array(grid_display)
    subplot_2.set_array(fast_grid_display)
    
    for p in big_ca.parameters():
        big_dtype = p.dtype
        
    for p in ca.parameters():
        default_dtype = p.dtype
        
    #for p in small_ca.parameters():
    small_dtype = fast_grid.dtype
        
        
    ax[0].set_title(f"dtype: {big_dtype}", fontsize=7)
    ax[1].set_title(f"dtype: {default_dtype}", fontsize=7)
    ax[2].set_title(f"dtype: {small_dtype}", fontsize=7)
    
    plt.tight_layout()

from copy import deepcopy

def update_fig_kernel_size(ii):

    global subplot_0    
    global subplot_1    
    global subplot_2
    global slow_grid
    global grid
    global fast_grid
    global ax
    global speedup_factor
    global scale_factor    
    
    dt = 1.0 *ca.dt
    
    default_ca_config = ca.make_config()
    default_radius = default_ca_config["neighborhood_kernel_config"]["radius"]
    
    big_ca_config = deepcopy(default_ca_config)
    big_ca_config["neighborhood_kernel_config"]["radius"] = int(default_radius * scale_factor)
    
    small_ca_config = deepcopy(default_ca_config)
    small_ca_config["neighborhood_kernel_config"]["radius"] =  int(default_radius // scale_factor)

    big_ca = CA()
    big_ca.load_config(big_ca_config)
    big_ca.no_grad()
    small_ca = CA()
    small_ca.load_config(small_ca_config)
    small_ca.no_grad()
    
    small_ca.dt = dt
    big_ca.dt = dt
    
    if ii == 0:
        plt.figure()
        plt.subplot(131)
        plt.imshow(big_ca.neighborhood_kernels.squeeze(), cmap="magma")
        plt.subplot(132)
        plt.imshow(ca.neighborhood_kernels.squeeze(), cmap="magma")
        plt.subplot(133)
        plt.imshow(small_ca.neighborhood_kernels.squeeze(), cmap="magma")
        plt.savefig("./scaled_kernels.png")
    
    for jj in range(speedup_factor):
        grid = ca(grid)
        slow_grid = big_ca(slow_grid)
        fast_grid = small_ca(fast_grid)
    
    grid_display = 1.0 - my_cmap(grid.cpu().squeeze())[:,:,:3]
    slow_grid_display = 1.0 - my_cmap(slow_grid.cpu().squeeze())[:,:,:3]
    fast_grid_display = 1.0 - my_cmap(fast_grid.cpu().squeeze())[:,:,:3]
    
    subplot_0.set_array(slow_grid_display)
    subplot_1.set_array(grid_display)
    subplot_2.set_array(fast_grid_display)
    
    big_radius = big_ca.make_config()["neighborhood_kernel_config"]["radius"] 
    small_radius = small_ca.make_config()["neighborhood_kernel_config"]["radius"] 
    
    ax[0].set_title(f"kernel radius: {big_radius:.4f}", fontsize=7)
    ax[1].set_title(f"kernel radius: {default_radius:.4f}", fontsize=7)
    ax[2].set_title(f"kernel radius: {small_radius:.4f}", fontsize=7)
    
    plt.tight_layout()


In [ ]:
torch.set_default_dtype(torch.float32)
my_device = torch.device("cpu")

In [ ]:
lib = Librarian()
for nb, elem in enumerate(lib.index):
    print(f"{nb}   {elem}")

# A small glider in _Scutium gravidus_ requires a large temporal step-size to maintain itself



$$

$$

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 20
step_size = 0.33
num_frames = 512
speedup_factor = 1
small_dim = 48
pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)

slow_grid = 1.0 * grid
fast_grid = 1.0 * grid 
fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])

# uncomment to save animation
#matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=20).save("../assets/dt_scutium.mp4")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=20).to_jshtml())

# Aproximating spatial continuity too closely in _Scutium gravidus_ 



$$

$$

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 20
step_size = 0.975
num_frames = 512
speedup_factor = 1
scale_factor = 5

pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)


slow_grid = torch.tensor(skimage.transform.rescale(grid.numpy().squeeze(), scale_factor, order=5)[None,None,:,:])

# to keep the kernel rescale and grid rescale the same
small_scale = (ca.kernel_radius // scale_factor) / ca.kernel_radius
fast_grid = torch.tensor(skimage.transform.rescale(grid.numpy().squeeze(), small_scale, order=5, anti_aliasing=True)[None,None,:,:] )

fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])
plt.show()

# uncomment to save animation
#matplotlib.animation.FuncAnimation(fig, update_fig_kernel_size, frames=num_frames, interval=20).save("../assets/kernel_sizes_scutium.mp4")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig_kernel_size, frames=num_frames, interval=20).to_jshtml())

# Too high a floating point precision can also make _Scutium_ unstable (sometimes)

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 20
step_size = 0.95
num_frames = 512
speedup_factor = 1
small_dim = 64

pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)


slow_grid = grid.to(torch.float64)


fast_grid = grid.to(torch.float16)

fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])
plt.show()

#plt.close("all")
#IPython.display.HTML(
#matplotlib.animation.FuncAnimation(fig, update_fig_precision, frames=num_frames, interval=20).save("../assets/dtypes_scutium.gif")
#.to_jshtml())

# NOTE: 
## The `float16` simulation uses a workaround on cpu: 
##     casting to float32 and back to float16 for every layer and math operation
##     Native float16 operations will only take place on compatible GPUs.
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig_precision, frames=num_frames, interval=20).to_jshtml())

# _Orbium_ behaves differently

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 11
step_size = 0.1
num_frames = 512
speedup_factor = 1
small_dim = 48
pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)

slow_grid = 1.0 * grid
fast_grid = 1.0 * grid 
fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])

# uncomment to save animation
#matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=20).save("../assets/dt_orbium.mp4")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=20).to_jshtml())

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 11
step_size = 0.25
num_frames = 512
speedup_factor = 1
scale_factor = 5

pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)


ca_config = ca.make_config()
ca_config["neighborhood_kernel_config"]["radius"] = 13
ca.load_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)


slow_grid = torch.tensor(skimage.transform.rescale(grid.numpy().squeeze(), scale_factor, order=5)[None,None,:,:])

# to keep the kernel rescale and grid rescale the same
small_scale = (ca.kernel_radius // scale_factor) / ca.kernel_radius
fast_grid = torch.tensor(skimage.transform.rescale(grid.numpy().squeeze(), small_scale, order=5, anti_aliasing=True)[None,None,:,:] )

fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])
plt.show()

# uncomment to save animation
#matplotlib.animation.FuncAnimation(fig, update_fig_kernel_size, frames=num_frames, interval=20).save("../assets/kernel_sizes_orbium.mp4")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig_kernel_size, frames=num_frames, interval=20).to_jshtml())

In [ ]:
#instantiate librarian
lib = Librarian()
lib.update_index()

pattern_index = 11
step_size = 0.25
num_frames = 512
speedup_factor = 1
small_dim = 64

pattern_name = lib.index[pattern_index]

#load scutium gravidus single glider
pattern, ca_meta = lib.load(lib.index[pattern_index])

ca_config, entry_point, commit_hash = ca_meta["ca_config"], ca_meta["entry_point"], ca_meta["commit_hash"]

ca = CA()
ca.restore_config(ca_config)
ca.no_grad()
ca.to_device(my_device)
ca.dt = step_size

dim_x = max([small_dim, pattern.shape[-2]*2])
dim_y = max([dim_x, pattern.shape[-1]*2])

grid = torch.zeros(1, 1, dim_x, dim_y)

offset_x = (dim_x - pattern.shape[-2]) // 2
offset_y = (dim_y - pattern.shape[-1]) // 2

grid[:,:,offset_x:offset_x+pattern.shape[-2], offset_y:offset_y+pattern.shape[-1]] = torch.tensor(pattern).clone()
grid = grid.to(my_device)


slow_grid = grid.to(torch.float64)


fast_grid = grid.to(torch.float32)

fig, ax = plot_grids(slow_grid, grid, fast_grid, titles=[f"{pattern_name} pattern, step size {ca.dt}, every {speedup_factor} frames"])
plt.show()

# uncomment to save animation
#matplotlib.animation.FuncAnimation(fig, update_fig_precision, frames=num_frames, interval=20).save("../assets/dtypes_orbium.mp4")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig_precision, frames=num_frames, interval=20).to_jshtml())